# mosi_openface_and_VED

Dans ce notebook :

Pour chaque vidéo .mp4 :  
- OpenFace produit un fichier CSV contenant :
- landmarks 2D 
- landmarks 3D
- pose de la tête
- regard (gaze)
- parfois Action Units (AU) 

PUIS,  

Extraction VED (Visual Emotional Description) :  
- À partir des landmarks du CSV, on simules certaines Action Units : AU01, AU02, AU07, AU12, AU25, AU26.
- on calcule une phrase décrivant l’expression (ex : “raised inner brows and parted lips”).
-Cette phrase est encodée via BERT → produit D_v (embedding 128d).


In [1]:

import os
import subprocess
import numpy as np
import pandas as pd
import torch

In [2]:


# -------------------------------------------------------------------
# Répertoires et chemins 
# -------------------------------------------------------------------

IDS_PATH = "MOSI_full/ids.txt"
VIDEO_DIR = "MOSI_full/video"
FEATURES_VIDEO_DIR = "MOSI_full/features_video"
VED_DIR = "MOSI_full/VED_features"

OPENFACE_EXE = "OpenFace_2.2.0_win_x64/FeatureExtraction.exe"

os.makedirs(FEATURES_VIDEO_DIR, exist_ok=True)
os.makedirs(VED_DIR, exist_ok=True)

# Import du TextEncoder depuis deva.py
from deva import TextEncoder

text_encoder = TextEncoder(bert_pretrained="bert-base-uncased")

# Chargement des IDs à traiter
with open(IDS_PATH, "r", encoding="utf-8") as f:
    ids_list = [line.strip() for line in f.readlines() if line.strip()]

print(f"Nombre d'IDs chargés depuis {IDS_PATH} :", len(ids_list))


c:\Users\elika\Documents\ECE ING5\Deep learning\dossier_deva\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nombre d'IDs chargés depuis MOSI_full/ids.txt : 93


In [3]:

def run_openface_for_id(vid_id):
    """Lance OpenFace pour un ID donné si le CSV n'existe pas encore."""
    csv_path = os.path.join(FEATURES_VIDEO_DIR, f"{vid_id}.csv")
    if os.path.exists(csv_path):
        print(f"✔ CSV déjà présent pour {vid_id}, on ne relance pas OpenFace.")
        return

    video_path = os.path.join(VIDEO_DIR, f"{vid_id}.mp4")
    if not os.path.exists(video_path):
        print(f"⚠ Vidéo introuvable pour {vid_id}: {video_path}")
        return

    cmd = [
        OPENFACE_EXE,
        "-f", video_path,
        "-out_dir", FEATURES_VIDEO_DIR,
        "-pose", "-gaze", "-2Dfp", "-3Dfp"  # flags courants, à adapter si besoin
    ]

    print(f"▶ Lancement OpenFace pour {vid_id} ...")
    try:
        # shell=False car on donne une liste (Windows gère bien ça en général)
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode != 0:
            print(f"❌ OpenFace a échoué pour {vid_id}")
            print("STDERR:", result.stderr[:500])
        else:
            print(f"✔ OpenFace OK pour {vid_id}")
    except Exception as e:
        print(f"❌ Exception lors de l'appel OpenFace pour {vid_id}: {e}")


In [4]:
# Boucle : génération / complétion des CSV OpenFace
for vid_id in ids_list:
    run_openface_for_id(vid_id)

✔ CSV déjà présent pour 03bSnISJMiM, on ne relance pas OpenFace.
✔ CSV déjà présent pour 0h-zjBukYpk, on ne relance pas OpenFace.
✔ CSV déjà présent pour 1DmNV9C1hbY, on ne relance pas OpenFace.
✔ CSV déjà présent pour 1iG0909rllw, on ne relance pas OpenFace.
✔ CSV déjà présent pour 2WGyTLYerpo, on ne relance pas OpenFace.
✔ CSV déjà présent pour 2iD-tVS8NPw, on ne relance pas OpenFace.
✔ CSV déjà présent pour 5W7Z1C_fDaE, on ne relance pas OpenFace.
✔ CSV déjà présent pour 6Egk_28TtTM, on ne relance pas OpenFace.
✔ CSV déjà présent pour 6_0THN4chvY, on ne relance pas OpenFace.
✔ CSV déjà présent pour 73jzhE8R1TQ, on ne relance pas OpenFace.
✔ CSV déjà présent pour 7JsX8y1ysxY, on ne relance pas OpenFace.
✔ CSV déjà présent pour 8OtFthrtaJM, on ne relance pas OpenFace.
✔ CSV déjà présent pour 8d-gEyoeBzc, on ne relance pas OpenFace.
✔ CSV déjà présent pour 8qrpnFRGt2A, on ne relance pas OpenFace.
✔ CSV déjà présent pour 9J25DZhivz8, on ne relance pas OpenFace.
✔ CSV déjà présent pour 9

In [5]:

def _normalize_scale(df):
    """Calcule une échelle de normalisation basée sur la distance menton-nez.
    Si indisponible, renvoie 1.0 pour éviter les crashes."""
    df = df.rename(columns={c: c.strip() for c in df.columns})
    try:
        x_chin, y_chin = df["X_8"], df["Y_8"]
        x_nose, y_nose = df["X_33"], df["Y_33"]
        dist = np.sqrt((x_chin - x_nose)**2 + (y_chin - y_nose)**2)
        dist_mean = float(np.mean(dist))
        return dist_mean if dist_mean > 1e-6 else 1.0
    except Exception:
        return 1.0


def compute_simulated_aus(df):
    """Calcule quelques AUs simulées à partir des landmarks 2D :
    - AU01: inner brow raise
    - AU02: outer brow raise
    - AU07: lid tightening
    - AU12: smile
    - AU25: lips part
    - AU26: jaw drop
    On renvoie un dict { 'AUxx': valeur_moyenne_normalisée }."""
    df = df.rename(columns={c: c.strip() for c in df.columns})
    scale = _normalize_scale(df)

    aus_values = { "AU01": [], "AU02": [], "AU07": [], "AU12": [], "AU25": [], "AU26": [] }

    for _, row in df.iterrows():
        try:
            y_brow_inner_left  = row["Y_21"]
            y_brow_inner_right = row["Y_22"]
            y_eye_left_top     = row["Y_37"]
            y_eye_right_top    = row["Y_43"]

            brow_elev_left  = (y_eye_left_top - y_brow_inner_left)  / scale
            brow_elev_right = (y_eye_right_top - y_brow_inner_right) / scale
            AU01_val = (brow_elev_left + brow_elev_right) / 2.0
            AU02_val = AU01_val

            y_eye_left_bottom  = row["Y_41"]
            y_eye_right_bottom = row["Y_47"]
            eye_open_left  = (y_eye_left_bottom - y_eye_left_top) / scale
            eye_open_right = (y_eye_right_bottom - y_eye_right_top) / scale
            AU07_val = 1.0 - ((eye_open_left + eye_open_right) / 2.0)

            x_mouth_left  = row["X_48"]
            x_mouth_right = row["X_54"]
            mouth_width = (x_mouth_right - x_mouth_left) / scale
            AU12_val = mouth_width

            y_mouth_top    = row["Y_62"]
            y_mouth_bottom = row["Y_66"]
            mouth_open = (y_mouth_bottom - y_mouth_top) / scale
            AU25_val = mouth_open
            AU26_val = mouth_open

            aus_values["AU01"].append(AU01_val)
            aus_values["AU02"].append(AU02_val)
            aus_values["AU07"].append(AU07_val)
            aus_values["AU12"].append(AU12_val)
            aus_values["AU25"].append(AU25_val)
            aus_values["AU26"].append(AU26_val)

        except KeyError:
            continue

    aus_mean = {}
    for au, vals in aus_values.items():
        if len(vals) == 0:
            aus_mean[au] = 0.0
        else:
            aus_mean[au] = float(np.mean(vals))

    return aus_mean


In [6]:

def visual_description(aus_dict):
    """Génère une phrase textuelle en anglais à partir d'AUs simulées."""
    threshold = 0.15
    active = [au for au, v in aus_dict.items() if v > threshold]

    if not active:
        return "The person shows a neutral facial expression."

    au_phrases = {
        "AU01": "raised inner brows",
        "AU02": "raised outer brows",
        "AU07": "tightened eyelids",
        "AU12": "a smiling expression",
        "AU25": "parted lips",
        "AU26": "a dropped jaw",
    }

    desc_list = [au_phrases.get(au, f"activation of {au}") for au in active]
    sentence = "The person shows signs of " + ", ".join(desc_list) + "."
    return sentence


def encode_VED_sentence(sentence):
    """Encode la phrase VED via le TextEncoder DEVA, renvoie D_v [1,8,128]."""
    tokenizer = text_encoder.bert.tokenizer

    encoded = tokenizer(
        sentence,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=50
    )

    bert_input = torch.stack(
        [
            encoded["input_ids"],
            encoded["attention_mask"],
            encoded["token_type_ids"]
        ],
        dim=1  # -> [1, 3, L]
    )

    with torch.no_grad():
        D_v = text_encoder(bert_input)

    return D_v


In [8]:

# Boucle : génération des VED_features à partir des CSV OpenFace

for vid_id in ids_list:
    csv_path = os.path.join(FEATURES_VIDEO_DIR, f"{vid_id}.csv")
    if not os.path.exists(csv_path):
        print(f"⚠ Pas de CSV pour {vid_id}, VED ignoré.")
        continue

    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"❌ Impossible de lire {csv_path}: {e}")
        continue

    df = df.rename(columns={c: c.strip() for c in df.columns})

    aus_dict = compute_simulated_aus(df)
    sentence = visual_description(aus_dict)
    print(f"VED phrase pour {vid_id}: {sentence}")

    D_v = encode_VED_sentence(sentence)

    out_path = os.path.join(VED_DIR, f"{vid_id}_VED.pt")
    torch.save(D_v, out_path)

    print("✔ Saved VED:", out_path)


VED phrase pour 03bSnISJMiM: The person shows signs of raised inner brows, raised outer brows, tightened eyelids, a smiling expression.
✔ Saved VED: MOSI_full/VED_features\03bSnISJMiM_VED.pt
VED phrase pour 0h-zjBukYpk: The person shows signs of raised inner brows, raised outer brows, tightened eyelids, a smiling expression.
✔ Saved VED: MOSI_full/VED_features\0h-zjBukYpk_VED.pt
VED phrase pour 1DmNV9C1hbY: The person shows signs of raised inner brows, raised outer brows, tightened eyelids, a smiling expression.
✔ Saved VED: MOSI_full/VED_features\1DmNV9C1hbY_VED.pt
VED phrase pour 1iG0909rllw: The person shows signs of raised inner brows, raised outer brows, tightened eyelids, a smiling expression.
✔ Saved VED: MOSI_full/VED_features\1iG0909rllw_VED.pt
VED phrase pour 2WGyTLYerpo: The person shows signs of raised inner brows, raised outer brows, tightened eyelids, a smiling expression.
✔ Saved VED: MOSI_full/VED_features\2WGyTLYerpo_VED.pt
VED phrase pour 2iD-tVS8NPw: The person shows